In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os import path


def compile_ranks(folder_path: str) -> pd.DataFrame:
    """ Compiles all csvs in the folder path into one df. """

    files = listdir(folder_path)

    dfs = []
    for file in files:
        full_path = path.join(folder_path, file)

        df = pd.read_csv(full_path)

        dfs.append(df)

    df = pd.concat(dfs)

    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year

    return df


def read_lookups(folder_path: str) -> pd.DataFrame:
    """ Reads in the draft/ranks data lookups as a df. """

    df = pd.read_csv(path.join(folder_path, '2022/lookups_2022.csv'))

    return df


def add_lookup_vals(df_base: pd.DataFrame, df_lookups: pd.DataFrame, lookup_type: str
                    , join_col_name: str, final_col_name: str) -> pd.DataFrame:
    """ 
    Adds the ranks_val from the df_lookups dataset to df_base based off the lookup_type
    and updates its name to the final_col_name.
    Point of this is for the player attributes in the drafts data to align with those
    in the ranks data.
    IMPORTANT: If other years ever end up being added, they must be all be found on
    the df passed to df_lookups. Otherwise, only the last year's values will be shown.
    """

    df_base = df_base.copy()
    df_lookups = df_lookups.loc[df_lookups['lookup_type'] == lookup_type].copy()

    df = pd.merge(df_base, df_lookups, how='left'
                , left_on=['draft_year', join_col_name]
                , right_on=['draft_year', 'drafts_val'])

    df.drop(columns=['lookup_type', 'drafts_val'], inplace=True)
    df.rename(columns={'ranks_val': final_col_name}, inplace=True)

    return df


def read_raw_data(folder_path):
    """
    Reads in the raw csvs and combines into one df.
    Might want to make more dynamic at some point.
    """

    df_raw_2021 = pd.read_csv(path.join(folder_path, '2021/raw_drafts.csv'))

    df_drafts_2022 = pd.read_csv(path.join(folder_path, '2022/df_drafts.csv'))
    df_league_info_2022 = pd.read_csv(path.join(folder_path, '2022/df_league_info.csv'))
    df_league_info_2022 = df_league_info_2022[['id', 'source', 'title']]

    rename_vars = {'id': 'draft_id', 'source': 'draft_source', 'title': 'draft_title'}
    df_league_info_2022.rename(columns=rename_vars, inplace=True)

    df_raw_2022 = pd.merge(df_drafts_2022, df_league_info_2022, how='left', on='draft_id')
    df_raw_2022

    df = pd.concat([df_raw_2021, df_raw_2022])

    df['full_name'] = df['first_name'] + ' ' + df['last_name']

    return df


def update_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    """ Updates columns to more appropriate dyptes. """
    
    # Replace null adps and update to float
    df['projection_adp'] = np.where(df['projection_adp'] == '-', 216, df['projection_adp'])
    df['projection_adp'] = df['projection_adp'].astype('float')

    # Update created_at to datetime to use as possible filter
    df['created_at'] = pd.to_datetime(df['created_at'], infer_datetime_format=True)

    return df


def drafts_w_player_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Filters out drafts which do not have player attributes (team, position, etc.)
    as these will likely serve as features for the model.
    """

    df = df_raw_all.copy()

    null_drafts = df.loc[df['first_name'].isnull()]

    null_drafts = null_drafts.drop_duplicates(subset='draft_id')['draft_id'].to_frame()
    null_drafts['ind_null_name_draft'] = 1

    df = pd.merge(df, null_drafts, on='draft_id', how='left')
    df = df.loc[df['ind_null_name_draft'].isnull()]

    df.drop(columns='ind_null_name_draft', inplace=True)

    return df


def _add_draft_dt(df: pd.DataFrame) -> pd.DataFrame:
    """ 
    Adds the datetime, date, and year of the draft.
    Note that created_at is datetime of each pick.
    """
    
    df_drafts = df[['draft_id', 'created_at']].copy()
    df_drafts.sort_values(by=['draft_id', 'created_at'], inplace=True)

    df_drafts.drop_duplicates(subset='draft_id', keep='first', inplace=True)
    df_drafts.rename(columns={'created_at': 'draft_datetime'}, inplace=True)

    df_drafts['draft_date'] = df_drafts['draft_datetime'].dt.normalize()
    df_drafts['draft_year'] = df_drafts['draft_datetime'].dt.year

    df = pd.merge(df, df_drafts, on='draft_id', how='left')

    return df


def add_draft_attrs(df: pd.DataFrame) -> pd.DataFrame:
    """ Adds draft level attributes. """

    # Adds number of teams by draft
    by_vars = ['draft_id', 'draft_entry_id']
    draft_teams = df[by_vars].drop_duplicates(subset=by_vars)

    num_teams = draft_teams.groupby('draft_id').size().to_frame('num_teams')

    df = pd.merge(df, num_teams, on='draft_id', how='left')

    # Adds round and pick of the round by draft
    df['round'] = ((df['number'] - 1) / df['num_teams']).astype('int') + 1
    df['round_pick'] = df['number'] - ((df['round'] - 1) * df['num_teams'])

    # Add datetime, date, and year of draft and year
    df = _add_draft_dt(df)

    return df


def add_ranks_lookups(df: pd.DataFrame) -> pd.DataFrame:
    """ 
    Adds the lookups required to map to the ranks df.
    IMPORATANT: Passed df must contain draft_year
    """

    df_lookups = read_lookups(DRAFTS_FOLDER)

    df = add_lookup_vals(df, df_lookups, 'player', 'full_name', 'final_player_name')
    df = add_lookup_vals(df, df_lookups, 'team', 'team_name', 'final_team_name')
    df = add_lookup_vals(df, df_lookups, 'position', 'position', 'final_position')

    # Draft date appears to be offset by a day relative to the ranks
    # for early morning drafts (or at least those with that timestamp).
    df['final_draft_date'] = np.where(df['draft_datetime'].dt.hour <= 5
                                    , df['draft_date'] - pd.Timedelta(days=1)
                                    , df['draft_date'])

    return df


def add_model_vars(df: pd.DataFrame) -> pd.DataFrame:
    """ Adds additional variables to test in the model. """

    df['actual_proj_adp_diff'] = df['projection_adp'] - df['number']

    return df


pd.set_option('display.max_rows', 300)

DRAFTS_FOLDER = '/home/cdelong/Python-Projects/UD-Draft-Model/Repo-Work/UD-Draft-Model/data'
RANKS_FOLDER = '/home/cdelong/Python-Projects/UD-Draft-Model/Repo-Work/UD-Draft-Model\
/data/2022/player_ranks'

df_ranks = compile_ranks(path.join(DRAFTS_FOLDER, '2022/player_ranks'))

df_raw_all = read_raw_data(DRAFTS_FOLDER)
df_updated_types = update_dtypes(df_raw_all)
df_complete_players = drafts_w_player_data(df_updated_types)
df_draft_attrs = add_draft_attrs(df_complete_players)
df_rank_lookups = add_ranks_lookups(df_draft_attrs)

df_final = add_model_vars(df_rank_lookups)

# df_final.dtypes

print(df_final.columns)
print(df_ranks.columns)

df_ranks

Index(['id', 'appearance_id', 'created_at', 'draft_entry_id', 'number',
       'pick_slot_id', 'points', 'projection_adp', 'projection_points',
       'swapped', 'draft_id', 'player_id', 'position', 'team_name',
       'first_name', 'last_name', 'draft_source', 'draft_title', 'full_name',
       'num_teams', 'round', 'round_pick', 'draft_datetime', 'draft_date',
       'draft_year', 'final_player_name', 'final_team_name', 'final_position',
       'final_draft_date', 'actual_proj_adp_diff'],
      dtype='object')
Index(['player', 'pos', 'team', 'adp', 'rank', 'date', 'Unnamed: 0', 'cdelong',
       'cdelong-hp.myfiosgateway.com', '17.10.2022 18:04',
       'file:///home/cdelong/.config/libreoffice/4;', 'year'],
      dtype='object')


,player,pos,team,adp,rank,date,Unnamed: 0,cdelong,cdelong-hp.myfiosgateway.com,17.10.2022 18:04,file:///home/cdelong/.config/libreoffice/4;,year
0,Jonathan Taylor,RB,IND,1.5,1.0,2022-08-19,NaN,NaN,NaN,NaN,NaN,2022
1,Christian McCaffrey,RB,CAR,1.9,2.0,2022-08-19,NaN,NaN,NaN,NaN,NaN,2022
2,Justin Jefferson,WR,MIN,3.3,3.0,2022-08-19,NaN,NaN,NaN,NaN,NaN,2022
3,Cooper Kupp,WR,LA,3.9,4.0,2022-08-19,NaN,NaN,NaN,NaN,NaN,2022
4,Ja'Marr Chase,WR,CIN,5.4,5.0,2022-08-19,NaN,NaN,NaN,NaN,NaN,2022
...,...,...,...,...,...,...,...,...,...,...,...,...
395,Dennis Houston,WR,DAL,216.0,396.0,2022-07-11,NaN,NaN,NaN,NaN,NaN,2022
396,Phillip Dorsett,WR,HOU,216.0,397.0,2022-07-11,NaN,NaN,NaN,NaN,NaN,2022
397,Todd Gurley,RB,NaN,216.0,398.0,2022-07-11,NaN,NaN,NaN,NaN,NaN,2022
398,Tim Tebow,TE,NaN,216.0,399.0,2022-07-11,NaN,NaN,NaN,NaN,NaN,2022


In [ ]:
""""
POTENTIAL TESTS:
    - Verify data types for imports
    - 
    - Any ranks lookup vals that are null? (i.e. does anything need added to the lookup file)
    - Does the draft ADP align with Ranks ADP when joined on player/position/date?

"""

In [ ]:
###########################################################################################
################################### Scratch to keep #######################################
###########################################################################################

In [ ]:
def validate_adp_ranks(df_drafts: pd.DataFrame, df_ranks: pd.DataFrame) -> pd.DataFrame:
    """ Verifies the adp from the drafts and ranks data is the same """
    
    df_base = df_drafts.copy()
    df_ranks = df_ranks.copy()

    keep_vars = ['draft_id', 'draft_datetime', 'created_at', 'final_player_name'
                , 'final_position', 'final_team_name'
                , 'final_draft_date', 'projection_adp']
    df_base = df_base[keep_vars]

    keep_vars = ['player', 'pos', 'team', 'date', 'adp']
    df_ranks = df_ranks[keep_vars]

    df = pd.merge(df_base, df_ranks, how='left'
                , left_on=['final_player_name', 'final_position', 'final_draft_date']
                , right_on=['player', 'pos', 'date'])

    df = df.loc[df['projection_adp'] != df['adp']]

    return df


def find_missing_lookups(df_drafts: pd.DataFrame, df_ranks: pd.DataFrame) -> pd.DataFrame:
    """
    Find players in the drafts data that do not map to the ranks data.
    Note that these will need added to the lookups file.
    """

    ranks_var = 'player'
    # drafts_var = 'full_name'
    drafts_var = 'final_player_name'

    df_ranks = df_ranks[[ranks_var]].drop_duplicates(subset=ranks_var)
    df_drafts = df_drafts[[drafts_var]].drop_duplicates(subset=drafts_var)

    df = pd.merge(df_drafts, df_ranks, how='left'
                    , left_on=drafts_var, right_on=ranks_var)

    df = df.loc[df['player'].isnull()]

    return df


_df_drafts = df_final.loc[df_final['draft_year'] == 2022].copy()
_df_ranks = df_ranks.copy()

df_adp_val = validate_adp_ranks(_df_drafts, _df_ranks)
df_missing_lookups = find_missing_lookups(_df_drafts, _df_ranks)


In [ ]:
###########################################################################################
################################### Basic Exploration #####################################
###########################################################################################

In [45]:
# df = df_final.copy()

dfs = []
for round in range(1, 19):
    df = df_final.loc[df_final['round'] == round]

    df = df['actual_proj_adp_diff'].quantile([.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]).to_frame()
    df = df.transpose()

    df['round'] = round

    cols = df.columns.to_list()
    cols = cols[-1:] + cols[:-1]

    df = df[cols]

    dfs.append(df)

df = pd.concat(dfs)


# df[['0.01', '0.05']]
df


,round,0.01,0.05,0.25,0.5,0.75,0.95,0.99
actual_proj_adp_diff,1,-3.300,-2.000,-0.400,0.2,1.100,3.500,7.000
actual_proj_adp_diff,2,-5.100,-3.600,-1.400,0.1,1.900,4.895,9.295
actual_proj_adp_diff,3,-6.700,-4.500,-1.500,0.4,2.500,7.195,11.357
actual_proj_adp_diff,4,-9.319,-5.900,-2.175,0.5,3.200,9.495,15.614
actual_proj_adp_diff,5,-11.538,-7.700,-3.100,0.0,3.475,9.895,18.214
actual_proj_adp_diff,6,-12.938,-8.300,-2.900,0.3,4.000,11.300,22.738
actual_proj_adp_diff,7,-13.619,-9.000,-3.600,0.1,4.400,12.300,23.495
actual_proj_adp_diff,8,-14.595,-10.300,-3.700,0.3,4.700,13.500,23.700
actual_proj_adp_diff,9,-17.200,-11.195,-4.000,1.2,5.500,14.300,22.733
actual_proj_adp_diff,10,-18.700,-12.695,-5.600,-0.4,5.300,17.195,33.300


In [121]:
# Check correlations between primary modeling variables and draft pick
df[['number', 'projection_adp', 'actual_proj_adp_diff']].corr()

,number,projection_adp,actual_proj_adp_diff
number,1.000000,0.989551,-0.024938
projection_adp,0.989551,1.000000,0.119459
actual_proj_adp_diff,-0.024938,0.119459,1.000000


In [70]:
df = df_complete_players.copy()
df = df[['draft_id', 'draft_source']].drop_duplicates(subset='draft_id')

df = df.groupby('draft_source', dropna=False).size().to_frame('num_drafts').reset_index()

df

,draft_source,num_drafts
0,sit_and_go,68
1,tournament,33
2,NaN,55
